#1-1

MNIST 데이터셋을 사용하여 간단한 GAN을 구현한 코드입니다.

코드를 실행시키고, 주석을 달아주세요.

In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
from torchvision.utils import save_image

In [2]:
# 하이퍼파라미터 설정
num_epochs = 100
batch_size = 100
learning_rate = 0.0002
img_size = 28 * 28
noise_size = 100
hidden_size1 = 256
hidden_size2 = 512
hidden_size3 = 1024
dir_name = "GAN_results"

# GPU 사용 여부 확인
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 결과 이미지를 저장할 디렉토리 생성
if not os.path.exists(dir_name):
    os.makedirs(dir_name)

# 전처리
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

# MNIST 데이터셋 로드
MNIST_dataset = datasets.MNIST(root='../../data/',
                               train=True,
                               transform=transform,
                               download=True)

# DataLoader 설정
data_loader = torch.utils.data.DataLoader(dataset=MNIST_dataset,
                                          batch_size=batch_size,
                                          shuffle=True)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:02<00:00, 4585226.99it/s]


Extracting ../../data/MNIST/raw/train-images-idx3-ubyte.gz to ../../data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 134182.23it/s]


Extracting ../../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../../data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:01<00:00, 1085383.04it/s]


Extracting ../../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../../data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 4363382.68it/s]

Extracting ../../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../../data/MNIST/raw



In [3]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(img_size, hidden_size3), # 입력 이미지 크기를 hidden_size3으로 변환
            nn.LeakyReLU(0.2), # Leaky ReLU 활성화 함수
            nn.Linear(hidden_size3, hidden_size2), # hidden_size3을 hidden_size2로 변환
            nn.LeakyReLU(0.2), # Leaky ReLU 활성화 함수
            nn.Linear(hidden_size2, hidden_size1), # hidden_size2를 hidden_size1로 변환
            nn.LeakyReLU(0.2), # Leaky ReLU 활성화 함수
            nn.Linear(hidden_size1, 1), # 최종 출력을 위한 선형 변환
            nn.Sigmoid() # Sigmoid 함수를 통해 0과 1 사이의 확률 값으로 변환
        )

    def forward(self, x):
        # 입력 x를 모델에 통과시켜 결과를 반환
        return self.model(x)

class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(noise_size, hidden_size1), # 입력 노이즈를 hidden_size1로 변환
            nn.ReLU(), # ReLU 활성화 함수
            nn.Linear(hidden_size1, hidden_size2), # hidden_size1을 hidden_size2로 변환
            nn.ReLU(), # ReLU 활성화 함수
            nn.Linear(hidden_size2, hidden_size3), # hidden_size2를 hidden_size3으로 변환
            nn.ReLU(), # ReLU 활성화 함수
            nn.Linear(hidden_size3, img_size), # hidden_size3을 이미지 크기로 변환
            nn.Tanh() # Tanh 함수를 통해 출력 범위를 -1과 1로 조정
        )

    def forward(self, x):
        # 입력 x를 모델에 통과시켜 결과를 반환
        return self.model(x)

# Discriminator와 Generator 모델 정의 및 GPU로 이동
discriminator = Discriminator().to(device)
generator = Generator().to(device)

# 손실 함수 정의
criterion = nn.BCELoss()
d_optimizer = optim.Adam(discriminator.parameters(), lr=learning_rate)
g_optimizer = optim.Adam(generator.parameters(), lr=learning_rate)


In [4]:
for epoch in range(num_epochs):
    for i, (images, _) in enumerate(data_loader): # 데이터 로더에서 배치 단위로 이미지 가져오기
        real_labels = torch.ones(batch_size, 1).to(device) # 실제 이미지에 대한 레이블 생성 (1)
        fake_labels = torch.zeros(batch_size, 1).to(device) # 가짜 이미지에 대한 레이블 생성 (0)

        real_images = images.reshape(batch_size, -1).to(device) # 이미지를 1차원으로 변환하여 GPU로 이동

        # Generator 훈련
        g_optimizer.zero_grad()

        z = torch.randn(batch_size, noise_size).to(device) # 랜덤 노이즈 생성
        fake_images = generator(z) # Generator로 가짜 이미지 생성

        g_loss = criterion(discriminator(fake_images), real_labels) # Generator의 손실 계산
        g_loss.backward()
        g_optimizer.step()

        # Discriminator 훈련
        d_optimizer.zero_grad()

        z = torch.randn(batch_size, noise_size).to(device) # 랜덤 노이즈 생성
        fake_images = generator(z) # Generator로 가짜 이미지 생성

        real_loss = criterion(discriminator(real_images), real_labels) # 실제 이미지에 대한 손실 계산
        fake_loss = criterion(discriminator(fake_images.detach()), fake_labels) # 가짜 이미지에 대한 손실 계산
        d_loss = (real_loss + fake_loss) / 2 # Discriminator의 손실 계산

        d_loss.backward()
        d_optimizer.step()

        # 150 스텝마다 손실 출력
        if (i + 1) % 150 == 0:
            print(f"Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{len(data_loader)}], "
                  f"D Loss: {d_loss.item():.4f}, G Loss: {g_loss.item():.4f}")

    # 생성된 가짜 이미지를 저장
    save_image(fake_images.view(batch_size, 1, 28, 28),
               os.path.join(dir_name, f'GAN_fake_image_{epoch + 1}.png'))

    # Discriminator의 성능 평가
    d_performance = discriminator(real_images).mean().item()
    g_performance = discriminator(fake_images).mean().item()
    print(f"---------Epoch [{epoch + 1}/{num_epochs}] : D Performance: {d_performance:.2f}, G Performance: {g_performance:.2f}")

Epoch [1/100], Step [150/600], D Loss: 0.0341, G Loss: 3.5758
Epoch [1/100], Step [300/600], D Loss: 0.2146, G Loss: 17.0593
Epoch [1/100], Step [450/600], D Loss: 0.0210, G Loss: 6.6702
Epoch [1/100], Step [600/600], D Loss: 0.0917, G Loss: 12.5119
---------Epoch [1/100] : D Performance: 0.97, G Performance: 0.00
Epoch [2/100], Step [150/600], D Loss: 0.0039, G Loss: 7.0128
Epoch [2/100], Step [300/600], D Loss: 0.1460, G Loss: 13.1850
Epoch [2/100], Step [450/600], D Loss: 0.2478, G Loss: 7.3226
Epoch [2/100], Step [600/600], D Loss: 0.1174, G Loss: 9.8547
---------Epoch [2/100] : D Performance: 0.90, G Performance: 0.01
Epoch [3/100], Step [150/600], D Loss: 0.3117, G Loss: 4.1842
Epoch [3/100], Step [300/600], D Loss: 0.3027, G Loss: 8.1486
Epoch [3/100], Step [450/600], D Loss: 0.2091, G Loss: 3.2236
Epoch [3/100], Step [600/600], D Loss: 0.8571, G Loss: 2.3695
---------Epoch [3/100] : D Performance: 0.55, G Performance: 0.35
Epoch [4/100], Step [150/600], D Loss: 0.3877, G Loss: 

#1-2

아래 마크다운으로 GAN_fake_image_1.png와 GAN_fake_image_100.png를 함께 첨부해주세요.

![GAN_fake_image_1.png](GAN_fake_image_1.png)
![GAN_fake_image_100.png](GAN_fake_image_100.png)